In [1]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="Qwen/Qwen3-Embedding-8B")

/root/my-volume/samsung-qreport-rag/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:11<00:00,  2.78s/it]


In [4]:
print(embeddings.embed_query("안녕"))

[0.02414531633257866, 0.007755632046610117, 0.010841292329132557, -0.022586124017834663, 0.009341023862361908, 0.0024616557639092207, -0.029239173978567123, 0.021518833935260773, -0.005622398108243942, 0.025139542296528816, -0.019726336002349854, -0.02231539785861969, 0.04515573009848595, -0.00304663204587996, -0.0016063901130110025, 0.014975042082369328, -0.03428233787417412, 0.00935528427362442, 0.03836595639586449, 0.04342006519436836, -0.009829171001911163, 0.010774693451821804, 0.02560797519981861, 0.0019250630866736174, 0.019886502996087074, 0.04503661394119263, -0.05025825649499893, 0.008627367205917835, -0.02842089720070362, 0.013753513805568218, -0.010856484062969685, -0.0009860479040071368, 0.009734616614878178, 0.021336181089282036, -0.017367400228977203, 0.022205593064427376, -0.0337570495903492, -0.00012894949759356678, 0.004492917098104954, -0.016506377607584, -0.006622200831770897, -0.0140994219109416, -5.970312122371979e-05, 0.00922464020550251, 0.0030695078894495964, -

In [10]:
import json
from langchain_core.documents import Document

with open("../../../data/parsed/pdfplumber.json", "r", encoding="utf-8") as f:
    docs_dict = json.load(f) 

docs = [Document(**d) for d in docs_dict]

print(docs[33].page_content)

나. 환경 관련 규제사항
당사는 법률에서 정하고 있는 각종 제품 환경규제와 사업장관리 환경규제를 철저하게 준수
하고 있습니다. 특히 정부의 저탄소 녹색성장 정책에 부응하여 관련 법규에서요구하는 사업
장에서 발생하는 온실가스 배출량과 에너지 사용량을 정부에 신고하고지속가능보고서 등을
통하여 이해관계자에게 관련 정보를 투명하게 제공하고 있습니다.
(제품 환경규제 준수)
전자제품은 일반적으로 소비자가 직접 휴대하거나, 가정에 설치하여 사용하고 있어 사용자
건강과 안전에 직ㆍ간접적인 영향을 미칠 수 있기 때문에 관련 환경규제가 강화되고 있습니
다. 이에 대해 당사는 부품 및 제품의 개발단계부터 제조, 유통, 사용, 폐기 등 제품의 전 과정
(Life Cycle)에 걸쳐 환경영향을 최소화하기 위한 활동을 하고있습니다. 유해물질을 제거한
부품 공급을 위해 협력회사 에코파트너 인증 제도를 운영하고 개발단계에서 제품의 친환경
요소(자원 사용 절감, 에너지 절약, 유해 물질 저감, 친환경소재 적용 등)를 제품에 반영하기
위해 에코디자인 평가 제도를 운영하며, 제품 사용 후 발생하는 폐전자제품을 회수하여 재활
용하는 폐제품 회수ㆍ재활용시스템을 유럽, 북미, 한국, 인도 등 각국에서 운영하고 있습니다
. 이러한 활동은 전기ㆍ전자 제품 관련 국내외 환경 법규를 준수할 뿐만 아니라 회사와 제품
의 차별화 요소로 기여하고 있습니다.
각국의 관련 법률은 다음과 같습니다.
1. 폐제품 회수ㆍ재활용법 (예: EU WEEE Directive)
2. 유해물질 사용 제한 (예: EU RoHS Directive, REACH Regulation)
3. 제품 에너지 소비효율 규정 (예: EU ErP Directive)
전자공시시스템 dart.fss.or.kr Page 31



In [11]:
from langchain_experimental.text_splitter import SemanticChunker

text_splitter = SemanticChunker(embeddings)

In [12]:
chunks = text_splitter.split_text(docs[33].page_content)

In [16]:
temp_a = text_splitter.split_documents(docs[:3])

In [19]:
print(temp_a[0])

page_content='목 차
분 기 보 고 서............................................................................................................................................1
【 대표이사 등의 확인 】...........................................................................................................................2
I. 회사의 개요............................................................................................................................................3
1.' metadata={'source': '../../data/raw/[삼성전자]분기보고서(2025.05.15).pdf', 'file_path': '../../data/raw/[삼성전자]분기보고서(2025.05.15).pdf', 'page': 0, 'total_pages': 302, 'Producer': 'iText® 5.4.0 ©2000-2012 1T3XT BVBA (AGPL-version)', 'ModDate': "D:20250515161427+09'00'", 'CreationDate': "D:20250515161427+09'00'"}
